In [5]:
def listToStr(MyList):
    text = ""
    for i in MyList:
        text += i + " , "

    return text

def listToStrBackwards(myString):
    list = myString.split(" , ")[:-1]
    return(list)




In [6]:
#Create SQL Database
#articles1 test
#articles2 all selcuk 
import sqlite3 as sql


SQLPATH = "scienedirect.db"

con = sql.connect(SQLPATH)
cursor = con.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS Articles2(
        status TEXT,
        title TEXT,
        link TEXT,
        subtype TEXT,
        author TEXT
    )""") 
cursor.execute("""CREATE TABLE IF NOT EXISTS ArtikelContent(
        title TEXT,
        abstract TEXT,
        abstract2 TEXT,
        indroduction TEXT,
        snippets TEXT,
        longText TEXT,
        author TEXT,
        authorAffiliation TEXT,
        requestDate TEXT,
        html TEXT
    )""")
con.commit() 


con.close()


In [7]:
import sqlite3
import pandas


con = sqlite3.connect("scienedirect.db")
cursor = con.cursor()

cursor.execute("SELECT * FROM ArtikelContent") 
rows = cursor.fetchall()

In [8]:
rowss = pandas.DataFrame(rows)
mytitle = "An investigation on failed or damaged reinforced concrete structures under their own-weight in Turkey"
rowss [rowss.iloc[:,1] == mytitle]

,0,1,2,3,4,5,6,7,8,9


!!!- -!!!
Selçuk Universitesine ait Sciencedirect Makale isim ve Link 



In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
 
import sqlite3,time

con = sqlite3.connect("scienedirect.db")
cursor = con.cursor()

driver = webdriver.Firefox("") 
driver.page_source


SHOW = 100 #tek seferde en fazla 100 Makale gösterilebilir
OFFSET = 0 #
PAGE = 0 #Başlangıç Sayfası

 


driver.get(f"https://www.sciencedirect.com/search?affiliations=selcuk&show={str(SHOW)}&offset={str(OFFSET)}")
_ = input("Yüklendi mi?")

while True:    
    LINK = f"https://www.sciencedirect.com/search?affiliations=selcuk&show={str(SHOW)}&offset={str(OFFSET)}" 
    driver.get(LINK)
    time.sleep(5)

    html_icerigi = driver.page_source
    soup = BeautifulSoup(html_icerigi, 'html.parser')
    articles = soup.find(class_='search-result-wrapper')  

    PAGE += 1
    OFFSET += SHOW

    if (articles == None):
        print("Es gibt das Gesuchte Artikel nicht!!!!")
        break

    for article in articles:
        try:
            STATUS = article.find(class_='OpenAccessArchive text-xs hor').text
            print(STATUS)
        except:
            pass

        try:
            TITLE = article.find("a").text
            print(TITLE)
        except:
            pass
            
    
        try:
            LINK = article.find("a")["href"]
            print(LINK)
        except:
            pass
            

        try:
            TRASH_BUT_INFO = soup.select('[class*="SubType hor"]')[0].text
            print(TRASH_BUT_INFO)
        except:
            pass
            

        try:
            AUTHORS = article.select('[class*="Authors hor"]')[0].text
            print(AUTHORS)
        except:
            pass
            

        try:
            cursor.execute("INSERT INTO articles2 VALUES (?,?,?,?,?)", (STATUS, TITLE, LINK,TRASH_BUT_INFO,AUTHORS))
            con.commit()
        except:
            print('SQL FEHLER !!!')

        


In [ ]:
import sqlite3 , time

from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

con = sqlite3.connect("scienedirect.db")
cursor = con.cursor()

cursor.execute("SELECT * FROM articles2") 
rows = cursor.fetchall()

driver = webdriver.Firefox("")
time.sleep(3)

count = 0

for row in rows[2060:]:
    try:    
        status = row[0]
        title = row[1]
        link = row[2]
        subtype = row[3]
        author = row[4]

        count=count+1

        driver.get(f"https://www.sciencedirect.com"+link)
        time.sleep(1)

        html_icerigi = driver.page_source
        soup = BeautifulSoup(html_icerigi, 'html.parser')    
    
        print("*********\nYeni Sayfa Yükleniyor\nCount: "+ str(count))

        #Navigation Bölümün Çıkartılması
        targets = soup.find_all(role="navigation")
        if targets:
            for target in targets:
                target.decompose()    

            targets = soup.find(role="navigation")
            if targets == None:
                print("Silme İşlemi Başarılıydı. Navigation tamamen silindi.")
        else:
            print("Navigation hiç bulunamadı. Bu yüzden de silinmesine gerek yok")

        #Article Bölümü var mı? 
        Article = soup.find_all(class_="Article")
        if Article == []:        
            print("Article bulunamadı")
            print("Bu Artikle Atlanmalı")

            #Artikel Bölümü yoksa ve error-card varsa client yeniden başlatılmalı
            if soup.find(class_="error-card"):
                raise ValueError("Client yeniden Başlatılmalı")
            
            continue
        else:
            print("Article Bulundu")



        
        #Yazarlar hakkında bilgi toplamak
        button = driver.find_elements(By.ID, "show-more-btn") 
        button[0].click()

        html_icerigi = driver.page_source
        soup = BeautifulSoup(html_icerigi, 'html.parser')    

        zAUTHORaffiliation = ''
        for i in soup.find_all(class_="affiliation"):
            r1 = i.find('dt').getText() if i.find('dt') else ' ' 
            r2 = i.find('dd').getText() if i.find('dd') else ' '
            zAUTHORaffiliation += r1+ ' -> ' +  r2 + '\n'            
      




        article = soup.find(class_="Article")

        zABSTRACT = article.find(id="preview-section-abstract") 
        zINSTRODUCTION = article.find(id="preview-section-introduction")
        zSNIPPETS = article.find(id="preview-section-snippets")

        zTITLE = article.find(id="screen-reader-main-title")    
        zABSTRACT2 = article.find(id="abstracts")
        zLONGText = article.find(id="body")
        
        zAUTHOR = article.find(class_="AuthorGroups").find_all("button") if (article and article.find(class_="AuthorGroups")) else None
        zAUTHOR = listToStr([author.text for author in zAUTHOR]) if zAUTHOR else None

        zABSTRACT = zABSTRACT.text if zABSTRACT else ""    
        zINSTRODUCTION = zINSTRODUCTION.text if zINSTRODUCTION else ""
        zSNIPPETS = zSNIPPETS.text if zSNIPPETS else ""

        zTITLE = zTITLE.text if zTITLE else ""
        zABSTRACT2 = zABSTRACT2.text if zABSTRACT2 else ""
        zLONGText = zLONGText.text if zLONGText else ""
        zAUTHOR = zAUTHOR if zAUTHOR else ""

        cursor.execute("INSERT INTO ArtikelContent VALUES (?,?,?,?,?,?,?,?,?,?)", (zTITLE, zABSTRACT, zABSTRACT2,zINSTRODUCTION,zSNIPPETS,zLONGText,zAUTHOR,zAUTHORaffiliation,time.time(),str(soup)))
        con.commit()

        #print("zABSTRACT*****\n", zABSTRACT,"\n","zINSTRODUCTION*****\n",zINSTRODUCTION,"\n","zSNIPPETS*****\n",zSNIPPETS,"\n","zTITLE*****\n",zTITLE,"\n","zABSTRACT2*****\n",zABSTRACT2,"\n","zLONGText*****\n",zLONGText,"\n","zAUTHOR*****\n",zAUTHOR,"\n",

    
    except ValueError as v:
        print(v)
        driver.close();     
        time.sleep(1)

        driver = webdriver.Firefox("")
        
    except:
        print("***********************************\n")
        print("Hatalı Count: ",count) 
        print("***********************************\n")  
    
    
 


In [1]:
import sqlite3 as sql
import pandas 

con = sql.connect("scienedirect.db")
cursor = con.cursor()

cursor.execute("SELECT * FROM ArtikelContent") 
rows = cursor.fetchall()

koalas = pandas.DataFrame(rows)
title = koalas.iloc[:,0]
abstract = koalas.iloc[:,1]
abstract2 = koalas.iloc[:,2]
introduction = koalas.iloc[:,3]
snippets = koalas.iloc[:,4]
longText = koalas.iloc[:,5]
author = koalas.iloc[:,6]
authorAffiliation = koalas.iloc[:,7]

In [ ]:
targetLetter=[]
trash=[]

t = authorAffiliation[1773].split("\n")
if ('' in t) : t.remove('')   
if len(t) == 1:
    print('Hepsi Selçuktandır -> 999')
    targetLetter.append('999')
else:
    for i in t:
        i = i.lower()
        if 'selcuk' in i:targetLetter.append(i[0])
        elif 'selçuk' in i:targetLetter.append(i[0])
        else: trash.append(i[0]) 

print('target: ' + ','.join(targetLetter))
print('trash: ' + ','.join(trash))

 

0       a -> Selcuk University, Faculty of Pharmacy, D...
1       a -> Department of Urology, Selcuk University ...
2       a -> Faculty of Nursing, Selcuk University, Ko...
3       a -> Computer Engineering, Selcuk University, ...
4       a -> Selcuk University, Department of Biochemi...
                              ...                        
4609      -> Graduate School of Pharmaceutical Science...
4610      -> Department of Food Science and Technology...
4611                                                     
4612    a -> Institutes for Systems Genetics, Frontier...
4613      -> Physiotherapy, Health Rehabilitation Scie...
Name: 7, Length: 4614, dtype: object

In [13]:
#Gerekli Func
def readPDF(path:str):
    from pypdf import PdfReader
    PATH_TO_PDF = path
    PDF_TEXT = ''

    reader = PdfReader(PATH_TO_PDF)

    for page in reader.pages:
        PDF_TEXT += page.extract_text()
        
    return(PDF_TEXT)

def combineTexts(*args):
    finalText = ' '.join(map(str,args))
    return finalText

def kelimeBulutu(text):
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image

    from wordcloud import WordCloud

    wc = WordCloud(
            background_color = 'black',
            contour_width = 2,
            contour_color = 'black',
            colormap = 'BuPu_r',
            width = 800,
            height = 500
        ).generate(text)
    plt.axis("off")
    plt.imshow(wc)

def kelimeSayacı(text):
    from collections import Counter
 
    kelimeler = text.lower().split()
    frekans = Counter(kelimeler)
    print(frekans)

def anahtarKelimeler(text):
    #2. Özel Kodlama: Anahtar Kelime Çıkartma 
    #Anahtar kelime çıkartma, metin analizinde önemli bir adımdır.
    #Kütüphane Önerisi: RAKE (Rapid Automatic Keyword Extraction), KeyBERT

    #pip install keybert
    from keybert import KeyBERT

    model = KeyBERT('distilbert-base-nli-mean-tokens')
    anahtar_kelime = model.extract_keywords(text, keyphrase_ngram_range=(1, 2),stop_words='english', top_n=3)
    print(anahtar_kelime)

def duyguAnalizi(text):
    from textblob import TextBlob
    from textblob import Word
     
    # Metni TextBlob ile analiz et     
    blob = TextBlob(text)
    print(f"Duygu Puanı: {blob.sentiment.polarity}")  # -1 ile 1 arasında bir değer
    print(f"Duygu Yoğunluğu: {blob.sentiment.subjectivity}")  # 0 ile 1 arasında bir değer

In [7]:
koalas.iloc[4,:][0]


'Encapsulation of diuron with Zn-based magnetic metal-organic framework and reduction of its mobility in soil'

In [8]:
from keybert import KeyBERT

text = "Deep Learning Approaches to Data Science"
anahtarKelimeler(text)

c:\Users\furka\Desktop\Metin Analizi 2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[('data science', 0.8135), ('deep learning', 0.6818), ('science', 0.658)]


In [19]:
anahtarKelimeler(title[0])

[('wound healing', 0.6019), ('nanofibers evaluation', 0.4655), ('functionalized calix', 0.3962)]


In [18]:
anahtarKelimeler(abstract[0])

[('biochemical data', 0.3809), ('biochemical analysis', 0.3582), ('extract pharmaceuticals', 0.3521)]


In [20]:
titles = koalas.iloc[:,1]
titleTEXT = ""

for textGroup in titles:
    for text in textGroup.split(" "):
        titleTEXT += " " + text
        
len(titleTEXT.split(" "))

746742

In [ ]:
titles = koalas.iloc[:,1]
titleTEXT = ""

for textGroup in titles:
    for text in textGroup.split(" "):
        titleTEXT += " " + text
        
len(titleTEXT.split(" "))

In [ ]:
print('\nKelime Sayacı:')
kelimeSayacı(titleTEXT)

print('\nDuygu Analizi:')
duyguAnalizi(titleTEXT)

kelimeBulutu(titleTEXT)